# Увеличение производительности Pandas: eval() и query()

## Основания для использования `query()` фтв `eval()`: составные выражения

**Далее используется метод `np.allclose`!!!**

Этот метод предназначен для поэлементного сравнения массивов с указанием допуска. Если допуска нет, то массивы должны быть полностью идентичны.

In [2]:
import numpy as np
rng = np.random.RandomState(42)
x = rng.rand(1000000)
y = rng.rand(1000000)
%timeit x + y

889 µs ± 21 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Такой подход хорошо работает и ускоряет вычисления, но становится менее эффективным в составных выражениях. Например, рассмотрим следующее выражение:

In [3]:
mask = (x > 0.5) & (y < 0.5)

Это выражение приблизительно эквивалентно такому:

```python
tmp1 = (x > 0.5)
tmp2 = (y < 0.5)
mask = tmp1 & tmp2
```

Другими словами, каждый промежуточный шаг явно алоцируется в памяти. Если x и y очень большие массивы, то расход памяти может быть очень большим.

In [7]:
import numexpr
mask_numexpr = numexpr.evaluate('(x > 0.5) & (y < 0.5)')
np.allclose(mask, mask_numexpr)

True

## `pandas.eval()` для эффективных операций

Функция Pandas `eval()` использует строковые выражения для эффективных вычислений с использованием `DataFrame`. Например, рассмотрим следующий `DataFrame`:

In [8]:
import pandas as pd
nrows, ncols = 100000, 100
rng = np.random.RandomState(42)
df1, df2, df3, df4 = (pd.DataFrame(rng.rand(nrows, ncols)) for i in range(4))

In [9]:
%timeit df1 + df2 + df3 + df4

264 ms ± 30.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Тоже самое, только с `pd.eval`:

In [10]:
%timeit pd.eval('df1 + df2 + df3 + df4')

73 ms ± 5.43 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


`eval()` получился в несколько раз быстрее!

### Операции поддерживаемые `pd.eval()`

In [11]:
df1, df2, df3, df4, df5 = (pd.DataFrame(rng.randint(0, 1000, (100, 3))) for i in range(5))

In [12]:
# арифметические
result1 = -df1 * df2 / (df3 + df4) - df5
result2 = pd.eval('-df1 * df2 / (df3 + df4) - df5')
np.allclose(result1, result2)

True

In [13]:
# операции сравнения
result1 = (df1 < df2) & (df2 <= df3) & (df3 != df4)
result2 = pd.eval('df1 < df2 <= df3 != df4')
np.allclose(result1, result2)

True

In [14]:
# битовые операции
result1 = (df1 < 0.5) & (df2 < 0.5) | (df3 < df4)
result2 = pd.eval('(df1 < 0.5) & (df2 < 0.5) | (df3 < df4)')
np.allclose(result1, result2)

True

In [16]:
# битовые операции с `and` и `or`
result3 = pd.eval('(df1 < 0.5) and (df2 < 0.5) or (df3 < df4)')
np.allclose(result1, result3)

True

In [17]:
# атрибуты объектов и индексы
result1 = df2.T[0] + df3.iloc[1]
result2 = pd.eval('df2.T[0] + df3.iloc[1]')
np.allclose(result1, result2)

True

## `DataFrame.eval()` для операций по столбцам

`DataFrame` имеет свой метод `eval()`, который работает схожим образом с `pd.eval()`. Преимущество метода `eval()` в том, что доступ к колонкам может быть получен по имени. Мы будем использовать следующий массив в качестве примера:

In [19]:
df = pd.DataFrame(rng.rand(1000, 3), columns=['A', 'B', 'C'])
df.head()

,A,B,C
0,0.375506,0.406939,0.069938
1,0.069087,0.235615,0.154374
2,0.677945,0.433839,0.652324
3,0.264038,0.808055,0.347197
4,0.589161,0.252418,0.557789


Используя `pd.eval()` для выражения выше, мы можем вычислить выражение с тремя колонками следующим образом:

In [20]:
result1 = (df['A'] + df['B']) / (df['C'] - 1)
result2 = pd.eval("(df.A + df.B) / (df.C - 1)")
np.allclose(result1, result2)

True

`DataFrame.eval()` позволяет сделать это так:

In [22]:
result3 = df.eval('(A + B) / (C - 1)')
np.allclose(result1, result3)

True

### Присваивание в `DataFrame.eval()`

In [23]:
df.head()

,A,B,C
0,0.375506,0.406939,0.069938
1,0.069087,0.235615,0.154374
2,0.677945,0.433839,0.652324
3,0.264038,0.808055,0.347197
4,0.589161,0.252418,0.557789


In [24]:
df.eval('D = (A + B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.375506,0.406939,0.069938,11.187620
1,0.069087,0.235615,0.154374,1.973796
2,0.677945,0.433839,0.652324,1.704344
3,0.264038,0.808055,0.347197,3.087857
4,0.589161,0.252418,0.557789,1.508776


### Локальные переменные в `DataFrame.eval()`

`DataFrame.eval()` поддерживает дополнительный синтаксис, который позволяет обращаться к локальным переменным Python (переменным в основном коде).

In [25]:
column_mean = df.mean(1)
result1 = df['A'] + column_mean
result2 = df.eval('A + @column_mean')
np.allclose(result1, result2)

True

## `DataFrame.query()` метод

Возьмем следующий пример:

In [26]:
result1 = df[(df.A < 0.5) & (df.B < 0.5)]
result2 = pd.eval('df[(df.A < 0.5) & (df.B < 0.5)]')
np.allclose(result1, result2)

True

In [27]:
result2 = df.query('A < 0.5 and B < 0.5')
np.allclose(result1, result2)

True

## Производительность: когда следует использовать эти функции

`eval()`/`query()` в среднем дают небольшой прирост производительности, а для небольших массивов могут быть даже медленнее, чем традиционный подход.

Основное преимущество этих методов в том, что они позволяют значительно сэкономить память.